# OptiGuide Example




Here we give a simple example, as designed and illustrated in the [OptiGuide paper](https://arxiv.org/abs/2307.03875).
While the original paper is designed specifically for supply chain optimization, the general framework can be easily adapted to other applications with coding capacity.




## OptiGuide for Supply Chain Optimization: System Design Overview

The original system design for OptiGuide, tailored for supply chain optimization, is presented below.

The collaboration among three agents -- Coder, Safeguard, and Interpreter -- lies at the core of this system. They leverage a set of external tools and a large language model (LLM) to address users' questions related to supply chain applications. For a comprehensive understanding of the design and data flow, detailed information can be found in the original [paper](https://arxiv.org/abs/2307.03875).


![optiguide system](https://www.beibinli.com/docs/optiguide/optiguide_system.png)


## New Implementation



![](new_design.png)

Advantages of this multi-agent design with autogen:
- Collaborative Problem Solving: The collaboration among the user proxy agent and the assistant agents fosters a cooperative problem-solving environment. The agents can share information and knowledge, allowing them to complement each other's abilities and collectively arrive at better solutions. On the other hand, the Safeguard acts as a virtual adversarial checker, which can perform another safety check pass on the generated code.

- Modularity: The division of tasks into separate agents promotes modularity in the system. Each agent can be developed, tested, and maintained independently, simplifying the overall development process and facilitating code management.

- Memory Management: The OptiGuide agent's role in maintaining memory related to user interactions is crucial. The memory retention allows the agents to have context about a user's prior questions, making the decision-making process more informed and context-aware.



In [2]:
# Install Required Packages
%pip install optiguide

Note: you may need to restart the kernel to use updated packages.


In [3]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from optiguide.optiguide import OptiGuideAgent

In [4]:
autogen.oai.ChatCompletion.start_logging()

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        }
    }
)

Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [5]:
# Get the source code of our coffee example
code_url = "https://raw.githubusercontent.com/niharikp/football_players_transfers_optimisation/main/model_script.py"
response  = requests.get(code_url)


# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open(code_url, "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

from gurobipy import Model, GRB, quicksum
import pandas as pd

# Cleaned dataset with pre-calculated performance metric
data_path = 'https://raw.githubusercontent.com/gmoellering/dabp/main/final_merged.csv' #Replace data path here
players_data = pd.read_csv(data_path)

#Constraint variables
team_var = '131' #FC Barcelona
budget_var = 150000000
.
.
.

            team_player_dict[team_id] = [player_id]
       


for team in team_player_dict.keys():
    print(team)
    selected_players = players_data[players_data['player_id'].isin(team_player_dict[team])]
    print(selected_players[['player_id', 'name', 'position', 'normalised_performance_score', 'market_value_in_eur']])




In [6]:
# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to use just one supplier for roastery 2?
Answer Code:
```python
z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
for s in suppliers:
    m.addConstr(x[s,'roastery2'] <= capacity_in_supplier[s] * z[s], "_")
```

----------
Question: What if there's a 13% jump in the demand for light coffee at cafe1?
Answer Code:
```python
light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
```
"""

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [7]:
%%capture
agent = OptiGuideAgent(
    name="OptiGuide Football Example",
    source_code=code,
    debug_times=1,
    example_qa="",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

Now, let's create our user questions.

In [10]:
user.initiate_chat(agent, message="Which players are in the optimal solution? Please return name along with their player_id")

user (to OptiGuide Football Example):

Which players are in the optimal solution? Please return name along with their player_id

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Get a list of selected players
selected_players_list = []

for v in model.getVars():
    if v.x > 0:  # If the player is chosen in the solution
        # Extract player_id from the variable name
        _, player_id, _ = v.varName.split('_')
        player_id = int(player_id)
        # Append the player_id to the list
        selected_players_list.append(player_id)

# Find corresponding player names in the dataset
selected_players = players_data[players_data['player_id'].isin(selected_players_list)][['player_id', 'name']]

selected_players
```

------------------------------------------------

#### Question 1

In [17]:
user.initiate_chat(agent, message="Which players should we select and why?")

user (to OptiGuide Football Example):

Which players should we select and why?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

To answer this question, we first need to access the decision variables of the optimized model and examine the variable values. If a variable value is one, it means that this player should be selected.

Here's how to check which players should be selected and why, based on their existing performance score and market value:

```python
selected_players_info = []

# Iterate over model variables
for v in model.getVars():
    # Check if the variable is selected in the solution
    if v.x > 0:
        # Parsing player id from variable's name
        player_id = int(v.varName.split('_')[1])
        
        # Getting player's info
        player_info = player

#### Question 2

In [19]:
user.initiate_chat(agent, message="We don't like player 1547, who should we hire if we remove him from consideration?")

user (to OptiGuide Football Example):

We don't like player 1547, who should we hire if we remove him from consideration?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Remove player 1547 from consideration
players_to_be_excluded.append(1547)

# Update dataset without player 1547
players_data = players_data[~players_data['player_id'].isin(players_to_be_excluded)]

# Run the optimization model again
model.optimize()

# Report the solution
team_player_dict = {}

for v in model.getVars():
    if v.x > 0:  # If the player is chosen in the solution
        # Extracting i and j from the variable name
        _, player_id, team_id = v.varName.split('_')

        team_id = int(team_id)
        player_id = int(player_id)

        # Check if the team is already in the dicti

#### Question 3

In [20]:
user.initiate_chat(agent, message="We definitely want player 38253, who else should we hire if he absolutely has to be in the lineup?")

user (to OptiGuide Football Example):

We definitely want player 38253, who else should we hire if he absolutely has to be in the lineup?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Include player 38253 constraint
player_to_be_included = [38253]
if player_to_be_included:
    for player in player_to_be_included:
        # Ensure that this player is included in team 131
        model.addConstr(x[(player, '131')] == 1, name=f"Included_{player}")

# Update the model with the new constraint
model.update()

# Solve the new model
model.optimize()
```

--------------------------------------------------------------------------------
OptiGuide Football Example (to safeguard):


--- Code ---
# Include player 38253 constraint
player_to_be_included = [38253]
if player_to_be

#### Question 4

In [21]:
user.initiate_chat(agent, message="Our budget decreased by 5 million dollars, how does that change which players we should try to trade for?")

user (to OptiGuide Football Example):

Our budget decreased by 5 million dollars, how does that change which players we should try to trade for?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Step 1: Decrease the budget by 5 million dollars 
BUDGET[team_var] -= 5e6

# Step 2: Modify the budget constraint for the team
model.remove(model.getConstrByName(f"Budget_{team_var}"))
model.addConstr(quicksum(players_data.loc[players_data['player_id'] == i, 'market_value_in_eur'].iloc[0] * x[(i, j)]
                         for i in players_data['player_id']) <= BUDGET[team_var], name=f"Budget_{team_var}")

# Step 3: Rerun the model
model.optimize()

team_player_dict = {}

# Step 4: Iterate through the variables to get the team's new lineup
for v in model.getVars():
    if v

#### Question 5

In [22]:
user.initiate_chat(agent, message="Our budget increased by 10 million dollars, how does that change which players we should try to trade for?")

user (to OptiGuide Football Example):

Our budget increased by 10 million dollars, how does that change which players we should try to trade for?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Updating our budget
BUDGET[team_var] += 10000000

# Updating the constraints in the model
for i, constr in enumerate(model.getConstrs()):
    if "Budget" in constr.ConstrName:
        model.remove(constr)
        
        # Budget constraints
        model.addConstr(quicksum(players_data.loc[players_data['player_id'] == i, 'market_value_in_eur'].iloc[0] * x[(i, team_var)]
                                for i in players_data['player_id']) <= BUDGET[team_var], name=f"Budget_{team_var}")

# Re-optimizing the model
model.optimize()

# Going through the new optimal solution
team

#### Question 6

In [25]:
user.initiate_chat(agent, message="Why should we select player 36139 instead of player 169880?")

user (to OptiGuide Football Example):

Why should we select player 36139 instead of player 169880?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Get the details of the players
player_1 = players_data[players_data['player_id'] == 36139]
player_2 = players_data[players_data['player_id'] == 169880]

# Get performance scores
player_1_score = player_1["normalised_performance_score"].values[0]
player_2_score = player_2["normalised_performance_score"].values[0]

# Get market values
player_1_value = player_1['market_value_in_eur'].values[0]
player_2_value = player_2['market_value_in_eur'].values[0]

# Show the results
print(f"Player 36139: Performance Score = {player_1_score}, Market Value = {player_1_value}")
print(f"Player 169880: Performance Score = {player_2_score}, 

#### Question 7

In [26]:
user.initiate_chat(agent, message="Who are some of the players with a similar performance score to player 38253 but are less expensive?")

user (to OptiGuide Football Example):

Who are some of the players with a similar performance score to player 38253 but are less expensive?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# target player_id
target_player_id = 38253

# get the performance score and market value of the target player
target_player_perf = players_data.loc[players_data['player_id'] == target_player_id, 'normalised_performance_score'].iloc[0]
target_player_value = players_data.loc[players_data['player_id'] == target_player_id, 'market_value_in_eur'].iloc[0]

# find players with similar performance score but less expensive
similar_perf_less_expensive = players_data[
    (players_data['normalised_performance_score'] >= target_player_perf * 0.9) &  # 10% less performance is still similar
    

#### Question 8

In [27]:
user.initiate_chat(agent, message="Which players should we select if we want to use our entire budget and only pick 3 players?")

user (to OptiGuide Football Example):

Which players should we select if we want to use our entire budget and only pick 3 players?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

If you want to use your entire budget but only select 3 players, you should add a constraint that limits the total number of players selected. The code below adds that constraint to the model.

Add it to the model initialization right after `model = Model()`, then optimize the model and print the selected players' details as before:

```python

# Limit the total number of selected players to 3
model.addConstr(quicksum(x[(i, j)] for i in players_data['player_id'] for j in TEAMS)
                    <= 3, 'NumberOfPlayers')

model.optimize()
``` 

Then the output provides you the three players optimized

#### Question 9

In [28]:
user.initiate_chat(agent, message="Which players should we select if we want to use our entire budget and only pick 3 players and one of them has to from the Attack position, one of them has to be from the Defender position, and one has to be from the Midfield position?")

user (to OptiGuide Football Example):

Which players should we select if we want to use our entire budget and only pick 3 players and one of them has to from the Attack position, one of them has to be from the Defender position, and one has to be from the Midfield position?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Change the Constraint variables
team_var = '131' #FC Barcelona
budget_var = 150000000
attack_needed = 1
defender_needed = 1
midfield_needed = 1
players_to_be_excluded = []
player_to_be_included = []

#Constants to enable decision for more than one team
TEAMS = [team_var] 
POSITIONS = players_data['position'].unique()
BUDGET = {team_var: budget_var}
NEEDS = {team_var: {"Attack": attack_needed, "Midfield": midfield_needed, "Defender": defender_needed

#### Question 10

In [29]:
user.initiate_chat(agent, message="Which players should we select if we want to use our entire budget and only pick 4 players, 2 fromfrom the Attack position and two from the Midfield position?")

user (to OptiGuide Football Example):

Which players should we select if we want to use our entire budget and only pick 4 players, 2 fromfrom the Attack position and two from the Midfield position?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Update the variables to reflect the new user inputs
budget_var = 150000000
attack_needed = 2
midfield_needed = 2
defender_needed = 0
BUDGET = {team_var: budget_var}
NEEDS = {team_var: {"Attack": attack_needed, "Midfield": midfield_needed, "Defender": defender_needed}}

# Recreate constraints
for j in TEAMS:

    # Budget constraints
    model.addConstr(quicksum(players_data.loc[players_data['player_id'] == i, 'market_value_in_eur'].iloc[0] * x[(i, j)]
                             for i in players_data['player_id']) <= BUDGE

In [30]:
user.initiate_chat(agent, message="Which players should we select if only want to select 3 players and we want to maximize our performance but also save as much money as possible?")

user (to OptiGuide Football Example):

Which players should we select if only want to select 3 players and we want to maximize our performance but also save as much money as possible?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
# Define the number of players to be selected
players_to_select = 3

# Constraint for number of players to select
model.addConstr(quicksum(x[(i, j)] for i in players_data['player_id'] for j in TEAMS) <= players_to_select, name='NumPlayers')

# Re-optimize the model with the new constraint
model.optimize()
```

--------------------------------------------------------------------------------
OptiGuide Football Example (to safeguard):


--- Code ---
# Define the number of players to be selected
players_to_select = 3

# Constraint for number o

In [38]:
user.initiate_chat(agent, message="Which players should we select if we only want players whose total_goals_2022 was at least 5?")

user (to OptiGuide Football Example):

Which players should we select if we only want players whose total_goals_2022 was at least 5?

--------------------------------------------------------------------------------
OptiGuide Football Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Football Example):

```python
players_data = players_data[players_data['total_goals_2022'] >= 5]
```

--------------------------------------------------------------------------------
OptiGuide Football Example (to safeguard):


--- Code ---
players_data = players_data[players_data['total_goals_2022'] >= 5]

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide Football Example):

SAFE

--------------------------------------------------------------------------------
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta